In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

%matplotlib inline 

In [3]:
#load data
mapping=pd.read_csv('comments_employee_mapping.csv').dropna()
likes=pd.read_csv('comments_likeability.csv').dropna()
attr=pd.read_csv('employee_attrition.csv').dropna()
hap=pd.read_csv('happiness_level.csv').dropna()

#read dates as pandas date-type
attr['lastParticipationDate']=pd.to_datetime(attr['lastParticipationDate'],infer_datetime_format=True)
mapping['commentDate']=pd.to_datetime(mapping['commentDate'],infer_datetime_format=True)
hap['voteDate']=pd.to_datetime(hap['voteDate'],infer_datetime_format=True)

#merge employee number and company alias into id
for d in [mapping,likes,attr,hap]:
    d['id']=d['employee'].map(str)+d['companyAlias']

#linearize dates
attr['ordinal']=[x.toordinal() for x in attr['lastParticipationDate']]
mapping['ordinal']=[x.toordinal() for x in mapping['commentDate']]
hap['ordinal']=[x.toordinal() for x in hap['voteDate']]

/Users/vaibhavnayel/anaconda/lib/python2.7/site-packages/dateutil/parser/_parser.py:1204: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/Users/vaibhavnayel/anaconda/lib/python2.7/site-packages/dateutil/parser/_parser.py:1204: UnknownTimezoneWarning: tzname CEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [5]:
l=np.load('likes_matrix_sparse_arranged.npy')[0].todense()
c=np.load('comment_matrix_sparse_arranged.npy')[0].todense()

In [7]:
data=np.hstack([l,c,pd.get_dummies(attr.companyAlias)])

X_train, X_test, y_train, y_test = train_test_split(
    data,attr.stillExists,stratify=attr.stillExists, test_size=0.33, random_state=42)


In [ ]:
'''
pca = PCA(n_components=500)
X_train=pca.fit_transform(X_train)
X_test=pca.transform(X_test)
'''

In [9]:
clf=LogisticRegression(C=.07)
clf.fit(X_train,y_train)
print 'train acc:',clf.score(X_train,y_train)
print 'test acc:',clf.score(X_test,y_test)

train acc: 0.904054054054054
test acc: 0.8717421124828533
